In [1]:
import pandas as pd
from collections import defaultdict
from datetime import datetime
from pytz import timezone

import sys, os
sys.path.append('../../Python packages')

from custom_functions import send_info_google_sheet #type: ignore

In [2]:
raw_data = pd.read_csv('Data/Raw_data/aAll_data/2024-06-17_18-59.csv')

fees = pd.read_csv('Data/Withdrawal_fees.csv')
fees.set_index('Exchange', inplace=True)

In [3]:
wallets = pd.read_csv('Data/Wallets.csv')

# Set "Exchange" column to index
for column_name in ['Exchange', 'Cryptocurrency', 'Network']:
    wallets[column_name] = wallets[column_name].str.lower()

In [4]:
# List of available exchanges. These names were extracted from cryptorank.io
# huobipro = HTX, mxc= MEXC
available_ex= ['binance', 'bybit-spot', 'okx', 'kucoin', 'huobipro', 'mxc', 'bitget', 'bingx', 'lbank', 'coinw', 'p2pb2b', 'bitmart',
               'phemex-spot', 'bigone', 'poloniex', 'coinex', 'btse', 'coinsbit', 'xeggex', 'bydfi']

threshold = 0.6

date = datetime.now(tz=timezone('US/Eastern'))
date = date.strftime("%Y-%m-%d_%H-%M")

In [5]:
# Remove strings from dictionaries
temp_function = lambda x: x.replace("defaultdict(<class 'list'>, ", "").replace(")","")

output = defaultdict(list)
for item in raw_data.index:
    data = temp_function(raw_data.loc[item, 'Arbitrage data'])
    data = eval(data)
    
    # Convert to DataFrame
    df = pd.DataFrame(data, index=raw_data.loc[item, 'Indexes'].split(","))

    coin = raw_data.loc[item, 'Cryptocurrency']
    if not os.path.exists(f"Data/Raw_data/{coin}"):
        os.mkdir(f"Data/Raw_data/{coin}")

    df.to_csv(f"Data/Raw_data/{coin}/{date}.csv")

    # Select only exchanges available
    rows = list(filter(lambda x: x in available_ex,  df.index))
    cols = list(filter(lambda x: x in available_ex,  df.columns))
    df = df.loc[rows, cols].copy()

    # If the sum == 0, it means that all values are lower than the threshold
    if (df >= threshold).sum().sum() != 0:
        
        # Delete columns where the first value is lower than the threshold.
        # Assumption: the values from cryptorank are sorted from the highest to lowest
        index_to_drop = [num for num in range(df.shape[1]) if df.iloc[0, num] <= threshold]

        df.drop(df.columns[index_to_drop], axis=1, inplace=True)
    
        # Flat cryptcurrency name, exchanges where to buy, sell and percentages
        cryptocurrency = [coin] * df.shape[0] * df.shape[1]
        buy = buy = sum([[element] * df.shape[1] for element in df.index], [])
        sell = list(df.columns) * df.shape[0]

        output['Cryptocurrency'].extend(cryptocurrency)
        output['Exchange to Buy'].extend(buy)
        output['Exchange to Sell'].extend(sell)
        output['Percentage'].extend(df.to_numpy().flatten().tolist())

In [6]:
# Convert output to DataFrame
result = pd.DataFrame(output)

In [7]:
# Replace names
result.replace({'binance':'Binance',    'bybit-spot':'Bybit', 'okx':'OKX',           'kucoin':'Kucoin',  'huobipro':'HTX', 'mxc':'MEXC',
                'bitget':'Bitget',      'bitmart':'BitMart',  'bingx':'BingX',       'lbank':'Lbank',    'coinw':'CoinW',  'p2pb2b':'P2B',
                'phemex-spot':'Phemex', 'bigone':'BigOne',    'poloniex':'Poloniex', 'coinex': 'CoinEx', 'btse':'BTSE',
                'coinsbit':'Coinsbit',  'xeggex':'Xeggex',    'bydfi':'BYDFi'}, inplace=True)

In [8]:
result.head()

,Cryptocurrency,Exchange to Buy,Exchange to Sell,Percentage
0,lido-staked-ether,HTX,MEXC,1.31
1,lido-staked-ether,HTX,OKX,1.29
2,lido-staked-ether,HTX,BingX,1.23
3,lido-staked-ether,HTX,Bybit,1.20
4,lido-staked-ether,HTX,Bitget,1.14


In [9]:
# Search withdrawal fees
for exchange, operation in zip(['Exchange to Buy', 'Exchange to Sell'], ['Buy', 'Sell']):
    temp = result.set_index(exchange).copy()
    temp[f'{operation}ing fee'] = fees['Taker fee']

    # Buying fee or Selling fee
    result[f'{operation}ing fee'] = temp.reset_index()[f'{operation}ing fee']

In [10]:
# Total withdrawal fees
result['Fees'] = result['Buying fee'] + result['Selling fee']

# Compute Expected profit
result['Expected profit'] = result['Percentage'] - result['Fees']

# Filter data only when the Expected profit is greater than the threshold
result = result[result['Expected profit'] >= threshold].copy()

In [11]:
result.head()

,Cryptocurrency,Exchange to Buy,Exchange to Sell,Percentage,Buying fee,Selling fee,Fees,Expected profit
0,lido-staked-ether,HTX,MEXC,1.31,0.2,0.1,0.3,1.01
1,lido-staked-ether,HTX,OKX,1.29,0.2,0.1,0.3,0.99
2,lido-staked-ether,HTX,BingX,1.23,0.2,0.1,0.3,0.93
3,lido-staked-ether,HTX,Bybit,1.20,0.2,0.1,0.3,0.90
4,lido-staked-ether,HTX,Bitget,1.14,0.2,0.1,0.3,0.84


In [12]:
def search_wallet(exchange_1, exchange_2, coin, df_wallets):
    """
    Inputs:
    exchange_1: Exchange from which the coins will be withdrawn (Source).
    coin: Coins to withdraw.
    exchange_2: Exchange that receives the coins.
    df_wallets: DataFrame with wallets
    """

    # Lowercase
    exchange_1 = exchange_1.lower()
    exchange_2 = exchange_2.lower()

    # Search available wallets of the first exchange
    condition_1 = (df_wallets['Exchange'] == exchange_1) & (df_wallets['Cryptocurrency'] == coin)
    source = df_wallets[condition_1].dropna().reset_index(drop=True)

    # Rename columns
    source.columns = [item+"_1" if item != 'Network' else item for item in source.columns]

    # Search available wallets of the second exchange
    condition_2 = (df_wallets['Exchange'] == exchange_2) & (df_wallets['Cryptocurrency'] == coin)
    target = df_wallets[condition_2].dropna().reset_index(drop=True)

    # Rename columns
    target.columns = [item+"_2" if item != 'Network' else item for item in target.columns]

    # Intersect both DataFrames and select the first row
    intersection = pd.merge(source, target, how='left', on='Network').dropna().reset_index(drop=True)

    return intersection.loc[0, 'Wallet_2'], intersection.loc[0, 'Network']

In [13]:
result[['Wtihdraw USDT to', 'Network USDT', 'Wtihdraw COIN to', 'Network COIN']] = ""

In [14]:
usdt_location = 'Phemex'

In [15]:
result[['Wtihdraw USDT to', 'Network USDT']] = result.apply(lambda x: search_wallet(usdt_location,
                                                                                    x['Exchange to Buy'],
                                                                                    'tether',
                                                                                    wallets), axis = 1, result_type='expand')

In [16]:
result[['Wtihdraw COIN to', 'Network COIN']] = result.apply(lambda x: search_wallet(x['Exchange to Buy'],
                                                                                    x['Exchange to Sell'],
                                                                                    x['Cryptocurrency'],
                                                                                    wallets), axis = 1, result_type='expand')

In [17]:
result.head()

,Cryptocurrency,Exchange to Buy,Exchange to Sell,Percentage,Buying fee,Selling fee,Fees,Expected profit,Wtihdraw USDT to,Network USDT,Wtihdraw COIN to,Network COIN
0,lido-staked-ether,HTX,MEXC,1.31,0.2,0.1,0.3,1.01,xxxxxxxhuobipro1,avax c-chain,xxxxxxxmxc6,network_steth
1,lido-staked-ether,HTX,OKX,1.29,0.2,0.1,0.3,0.99,xxxxxxxhuobipro1,avax c-chain,xxxxxxxokx6,network_steth
2,lido-staked-ether,HTX,BingX,1.23,0.2,0.1,0.3,0.93,xxxxxxxhuobipro1,avax c-chain,xxxxxxxbingx6,network_steth
3,lido-staked-ether,HTX,Bybit,1.20,0.2,0.1,0.3,0.90,xxxxxxxhuobipro1,avax c-chain,xxxxxxxbybit6,network_steth
4,lido-staked-ether,HTX,Bitget,1.14,0.2,0.1,0.3,0.84,xxxxxxxhuobipro1,avax c-chain,xxxxxxxbitget6,network_steth


In [18]:
# Select 'Cryptocurrency' and 'Symbol' from wallet DataFrame
symbols = wallets[['Cryptocurrency', 'Symbol']].copy().drop_duplicates()

# Set "Cryptocurrency" as index in both DataFrames
symbols.set_index('Cryptocurrency', inplace=True)
result.set_index('Cryptocurrency', inplace=True)

result.loc[:, 'Symbol'] = symbols['Symbol']

result.reset_index(inplace=True)

In [19]:
# Delete wallet if the location of the USDT is the same of the exchange to buy.
result.loc[result['Exchange to Buy']==usdt_location, ['Wtihdraw USDT to', 'Network USDT']] = ""

In [20]:
result[['Wtihdraw USDT to', 'Network USDT', 'Exchange to Buy', 'Cryptocurrency', 'Symbol',
        'Wtihdraw COIN to', 'Network COIN', 'Exchange to Sell', 'Expected profit']]

,Wtihdraw USDT to,Network USDT,Exchange to Buy,Cryptocurrency,Symbol,Wtihdraw COIN to,Network COIN,Exchange to Sell,Expected profit
0,xxxxxxxhuobipro1,avax c-chain,HTX,lido-staked-ether,stETH,xxxxxxxmxc6,network_steth,MEXC,1.01
1,xxxxxxxhuobipro1,avax c-chain,HTX,lido-staked-ether,stETH,xxxxxxxokx6,network_steth,OKX,0.99
2,xxxxxxxhuobipro1,avax c-chain,HTX,lido-staked-ether,stETH,xxxxxxxbingx6,network_steth,BingX,0.93
3,xxxxxxxhuobipro1,avax c-chain,HTX,lido-staked-ether,stETH,xxxxxxxbybit6,network_steth,Bybit,0.90
4,xxxxxxxhuobipro1,avax c-chain,HTX,lido-staked-ether,stETH,xxxxxxxbitget6,network_steth,Bitget,0.84
5,xxxxxxxxeggex2,bsc (bep20),Xeggex,shiba-inu,SHIB,xxxxxxxmxc11,network_shib,MEXC,1.18
6,xxxxxxxxeggex2,bsc (bep20),Xeggex,shiba-inu,SHIB,xxxxxxxokx11,network_shib,OKX,1.16
7,xxxxxxxxeggex2,bsc (bep20),Xeggex,shiba-inu,SHIB,xxxxxxxbingx11,network_shib,BingX,1.13
8,xxxxxxxxeggex2,bsc (bep20),Xeggex,shiba-inu,SHIB,xxxxxxxkucoin11,network_shib,Kucoin,1.09


In [21]:
columns_order = ['Wtihdraw USDT to', 'Network USDT', 'Exchange to Buy', 'Cryptocurrency', 'Symbol',
                    'Wtihdraw COIN to', 'Network COIN', 'Exchange to Sell', 'Expected profit']

result[columns_order].sort_values(by='Expected profit', ascending=False).to_csv(f'Data/Cleaned_data/{date}.csv', index=False)


In [22]:
send_info_google_sheet(result[columns_order].sort_values(by='Expected profit', ascending=False),
                       "https://docs.google.com/spreadsheets/d/1SV1uBmX--LqyQHrwGzEbU7Z5h_NiIKdVKgkSZYWDldo/edit?usp=sharing",
                       'Data')

Sending data to Google Sheet.

The sheet already exist: {'code': 400, 'message': 'Invalid requests[0].addSheet: A sheet with the name "Data" already exists. Please enter another name.', 'status': 'INVALID_ARGUMENT'}
Data was succesfully saved to Google Sheet.
